# Generating Synthetic Datasets for Evaluating Retrieval Augmented Generation Systems


As Retrieval Augmented Generation (RAG) systems become more prevalent, evaluating their performance is essential to ensure quality and performace. However, collecting real-world data for evaluation can be costly and time-consuming, especially in the early stages of a project. To addresses this challenge of data scarcity, synthetic dataset generation provides a practical solution for generating datasets that mimic real human interactions, enabling efficient and scalable evaluation of RAG systems. By leveraging large language models and knowledge retrieval context, the proposed approach ensures that the synthetic datasets are diverse, realistic, and representative of real-world scenarios. This solution is relevant for developers and researchers working on RAG systems, as it streamlines the evaluation process and accelerates the iterative development cycle, ultimately leading to better-performing AI systems. The process of generating synthetic datasets is integrated in open source tools like [RAGAS](https://docs.ragas.io/en/stable/) and will be outlined in this notebook.  

In this notebook you will be guided through generating a synthetic dataset for a QA-RAG application using Anthropic Claude via the Bedrock API, Python and Langchain. The notebook consists of the following chapters: 

1. [Set-up of the environment](#1.-Set-up-of-the-environment)
2. [Loading and preparing context data](#2-loading-and-preparing-data)
3. [Initial Question Generation](#3.-Initial-Question-Generation)
4. [Answer Generation](#4.-Answer-Generation)
5. [Extracting Relevant Context](#5.-Extracting-Relevant-Context)
6. [Evolving Questions to fit End-User behaviour](#6.-Evolving-Questions-to-fit-end-users-behaviour)
7. [Automated Dataset Generation](#7.-Automated-Dataset-Generation)
8. [Assessing the questions quality using Critique Agents](#8-assessing-the-questions-quality-using-critique-agents)



## 1. Set-up of the environment
Let's start by installing the required libraries.

In [ ]:
%pip install -q langchain==0.1.10 boto3 pypdf

## 2. Loading and Preparing Data

For this lab you will use a fictuous use case where you want to build a chatbot to answer questions about Amazon shareholder letters. A typical technique to build such a chatbot is  Retrieval-Augmented Generation (RAG). While this lab focuses on dataset generation, let's start with a quick RAG primer for some background context. 

#### What is RAG?

Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. 

Now in order to build a synthetic training dataset for such a question answering RAG system, raw data from the knowledge source is used to derive possible user questions. For our use case you will use PDF files of shareholder letters with text information loaded from the interent to serve as the knowledge base. In production grade RAG implementations, the knowledge retriever may leverage a database that supports vector searches to dynamically lookup relevant documents that serve as the knowledge source.

In our case let's start by downloading the shareholder letters.

In [ ]:
# Import necessary libraries for downloading files
from urllib.request import urlretrieve 
import os

# Create folder to store downloaded files
# Use descriptive folder name relating to data
folder_name = "synthetic_dataset_generation"  

# Check if folder already exists, if yes do nothing
# If no, create the folder
if os.path.exists(folder_name):
    pass  
else:
    os.mkdir(folder_name)

# List of URLs of files to download
files = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',  
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

# Iterate through list of URLs 
for url in files:

    # Get file name from URL to use as local file name
    file_path = os.path.join("synthetic_dataset_generation", url.rpartition("/")[2])  
    urlretrieve(url, file_path)

Now that the context data in the form of the shareholder letters has been downloaded you now load PDF documents from the created directory, you will now split them into smaller text chunks using a recursive character text splitter from the Langchain library. The RecursiveCharacterTextSplitter divides the text into chunks of a specified size while allowing for overlap to prevent cutting sentences in half. When setting the chunk size, make sure it fits into the context window of your LLM and feel free to experiment with different chunk sizes.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders.pdf import PyPDFLoader, PyPDFDirectoryLoader


# Load PDF documents from directory
loader = PyPDFDirectoryLoader("./synthetic_dataset_generation/")  
documents = loader.load()

# Use recursive character splitter, works better for this PDF data set
text_splitter = RecursiveCharacterTextSplitter(

    # Split documents into small chunks
    chunk_size = 1500,  

    # Overlap chunks to reduce cutting sentences in half
    chunk_overlap  = 100,
    separators=["\n\n", "\n", ".", " ", ""],

)


# Split loaded documents into chunks
docs = text_splitter.split_documents(documents)

Let's have a look at the size of our data

In [ ]:
# Print metadata of the loaded documents
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} pages loaded is {avg_char_count_pre} characters.')
print(f'After the split you have {len(docs)}')
print(f'Average length among {len(docs)} chunks is {avg_char_count_post} characters.')

> **_NOTE:_**  As Amazon Bedrock will be used for generating synthetic data in the following you will connect to the Bedrock API. Further in the Lab you will use the Langchain library to communicate with the Amazon Bedrock API

In [ ]:
# Set up Amazon Bedrock as LLM supplier for synthetic dataset creation
import json
import os
import sys
import boto3

# set up a Bedrock-runtime client for inferencing large language models
boto3_bedrock = boto3.client('bedrock-runtime')

# set up a Bedrock client for performing administrative API calls
boto3_bedrock_admin = boto3.client('bedrock')


In [ ]:
# Model Selection
# Choosing claude 3 Haiku due to cost and performance efficiency
claude_3_haiku = "anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
# Set-up langchain LLM for implementing the synthetic dataset generation logic
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models import BedrockChat

# for each model provider there are different parameters to define when inferencing against the model
inference_modifier = {
                        "max_tokens": 10000,
                        "temperature": 0.5,
                        "top_k": 250,
                        "top_p": 1,
                    }
                     

llm = BedrockChat(model_id = claude_3_haiku,
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )

## 3. Initial Question Generation

As a first step you generate sample questions. You can use each of the generated chunks to generate synthetic questions that a real chatbot user might ask. You will prompt the LLM to analyze a chunk of shareholder letter data and generate a relevant question based on the information presented in the context. Below is a sample prompt to generate a question given a specific context. Note you are hardcoding to generate a single question for simplicity, of course you can also ask the LLM to generate multiple questions with a single prompt.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template to generate a question a end-user could have about a given context
initial_question_prompt_template = PromptTemplate(
    input_variables=["context"],
    template="""
    <Instructions Structure>
    1. Provide context
    2. Explain the task and rules
    3. Instruct to generate a question based on the context following the rules
    4. Specify output format
    </Instructions Structure>

    <Instructions>
    Here is some context:
    <context>
    {context}
    </context>

    Your task is to generate 1 question that can be answered using the provided context, following these rules:

    <rules>
    1. The question should make sense to humans even when read without the given context.
    2. The question should be fully answered from the given context.
    3. The question should be framed from a part of context that contains important information. It can also be from tables, code, etc.
    4. The answer to the question should not contain any links.
    5. The question should be of moderate difficulty.
    6. The question must be reasonable and must be understood and responded by humans.
    7. Do not use phrases like 'provided context', etc. in the question.
    8. Avoid framing questions using the word "and" that can be decomposed into more than one question.
    9. The question should not contain more than 10 words, make use of abbreviations wherever possible.
    </rules>

    To generate the question, first identify the most important or relevant part of the context. Then frame a question around that part that satisfies all the rules above.

    <thinking>
    [This is a space for you to write down your thoughts and identify relevant parts of the context as you formulate the question.]
    </thinking>

    Output only the generated question with a "?" at the end, no other text or characters.
    </Instructions>
    
    """)

def generate_question(doc, llm):

    # Pass in values to the input variables
    initial_question_prompt = initial_question_prompt_template.format(context=doc)
    
    initial_question = llm.invoke(initial_question_prompt)
    
    return initial_question

In [ ]:
# generate a question based on a given context
question = generate_question(docs[1], llm)
print(f"Intial question: {question.content}")

In [ ]:
print(question.content)

## 4. Answer Generation
To use the questions for evaluation you need to generate a reference answer for each of the questions to test against. Let's do this using following prompt template:

In [ ]:
# Create a prompt template that takes into consideration the the question and generates an answer
answer_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    <Instructions Structure>
    1. Provide context
    2. State the task and rules
    3. Provide the question
    4. Instruct to generate the answer based only on the given context
    5. Instruct to output only the generated answer sentence
    </Instructions Structure>

    <Instructions>
    <Task>
    <role>You are an experienced QA Engineer for building large language model applications.</role>
    <task>It is your task to generate an answer to the following question <question>{question}</question> only based on the <context>{context}</context></task>
    The output should be only the answer generated from the context.

    <rules>
    1. Only use the given context as a source for generating the answer.
    2. Be as precise as possible with answering the question.
    3. Be concise in answering the question and only answer the question at hand rather than adding extra information.
    </rules>

    Only output the generated answer as a sentence. No extra characters.
    </Task>
    </Instructions>
    
    Assistant:""")

def generate_answer(question: str, doc, llm):
    
    answer_prompt = answer_prompt_template.format(question = question, context=doc)
    
    answer = llm.invoke(answer_prompt)
    
    return answer

In [ ]:
answer = generate_answer(question, docs[1], llm)
print(f"Intial question: {question.content}")
print("---")
print(f"Reference Answer: {answer.content}")

## 5. Extracting Relevant Context
To make the dataset verifiable you use the following prompt to extract the relevant sentences from the given context to answer the generated question. Knowing the relevant sentences you can easyly check whether the question and answer are correct. 

In [ ]:
# To check whether an answer was correctly formulated by the large language model you get the relevant text passages from the documents used for answering the questions.
source_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""Human:
    <Instructions Structure>
    1. Provide the context
    2. State the task of extracting relevant sentences from the context to answer the question
    3. Provide the question
    4. Instruct to output only the relevant sentences, without any extra characters or explanations
    </Instructions Structure>

    <Instructions>
    Here is the context:
    <context>
    {context}
    </context>

    Your task is to extract the relevant sentences from the given context that can potentially help answer the following question. You are not allowed to make any changes to the sentences from the context.

    <question>
    {question}
    </question>

    Output only the relevant sentences you found, one sentence per line, without any extra characters or explanations.
    </Instructions>
    Assistant:""")

def generate_source(question: str, doc, llm):
        
    source_prompt = source_prompt_template.format(question = question, context=doc)
    
    source = llm.invoke(source_prompt)
    
    return source

In [ ]:
source_sentence = generate_source(question, docs[1], llm)
print(f"Intial question: {question.content}")
print("---")
print(f"Reference Answer: {answer.content}")
print("---")
print(f"Source Sentence: {source_sentence.content}")

## 6. Evolving Questions to fit end-users behaviour
When generating question & answer pairs from the same prompt for the whole dataset it might appear that the questions are repetetive, similar in form and thus not mimic real enduser behaviour. In this section you evolve the existing generated question to for example make it shorter and more precise. The prompt for generating questions that fit your use case heavily depend on your use case and thus your prompt must reflect your endusers by for instance setting the rules accordingly or by providing examples.

In [ ]:
# To generate a more versatile testing dataset you alternate the questions to see how your RAG systems performs against differently formulated of questions
question_compress_prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""
    <Instructions Structure>
    1. Provide context on the task
    2. State the rules for rewriting the question
    3. Instruct to output only the rewritten question with a question mark
    </Instructions Structure>

    <Instructions>
    <role>You are an experienced linguistics expert for building testsets for large language model applications.</role>

    <task>It is your task to rewrite the following question in a more indirect and compressed form, following these rules:

    <rules>
    1. Make the question more indirect
    2. Make the question shorter
    3. Use abbreviations if possible
    </rules>

    <question>
    {question}
    </question>

    Your output should only be the rewritten question with a question mark "?" at the end. Do not provide any other explanation or text.
    </task>
    </Instructions>
    
    """)


def compress_question(question): 
    # Pass in values to the input variables
    question_compress_prompt = question_compress_prompt_template.format(question=question)
    
    question_compressed = llm.invoke(question_compress_prompt)
        
    return question_compressed

In [ ]:
compressed_question = compress_question(question)
print(f"Intial question: {question.content}")
print("---")
print(f"Reference Answer: {answer.content}")
print("---")
print(f"Source Sentence: {source_sentence.content}")
print("---")
print(f"Compressed Question: {compressed_question.content}")


## 7. Automated Dataset Generation
To scale the process of the dataset generation you iterate over all chunks of your context, generate questions, answers, relevant sentences and evolutions for each chunk and save them to a pandas dataframe. 

In [ ]:
# Creating a subset of the loaded documents for lightweight testing. For generating data for every document please work with docs.
docs_subset = docs[0:2]

In [ ]:
from langchain_core.documents.base import Document

def generate_qa_dataset_doc(doc: Document, llm, dataset, doc_number):
    """A function to create a test dataset of questions for a given Document(Langchain Document type)"""
    
    # generate the initial question for the RAG testdataset
    question = generate_question(doc, llm)
    dataset.at[doc_number, "question"] = question.content
    
    # generate compressed  question to variate the dataset
    compressed_question = compress_question(question)
    dataset.at[doc_number, "question_compressed"] = compressed_question.content
   
    
    answer = generate_answer(question, doc, llm)
    dataset.at[doc_number, "reference_answer"] = answer.content
        
    source_sentence = generate_source(question, doc, llm)
    dataset.at[doc_number, "source_sentence"] = source_sentence.content
    
    source_raw = doc
    dataset.at[doc_number, "source_raw"] = source_raw.page_content
    
    source_document = doc.metadata["source"]
    dataset.at[doc_number, "source_document"] = source_document
    
    
    return dataset
    

In [ ]:
# create a dataset class that in the end can be used to generate the dataset
import pandas as pd
import time

dataset = pd.DataFrame(columns=["question", "question_compressed", "reference_answer", "source_sentence","source_raw","source_document" ])        

In [ ]:
from langchain_core.documents.base import Document
from tqdm import tqdm

def generate_dataset(documents: Document,llm, dataset):

    print(f"start generating dataset from {len(documents)} docuements")
    print("---")
    generation_time_start = time.time()
    
    for doc in tqdm(range(len(documents))):
        q_generation_time_start = time.time()
        dataset = generate_qa_dataset_doc(doc = documents[doc], llm = llm, dataset = dataset, doc_number = doc)
        q_generation_time_end = time.time()
        total_elapsed_time_generation = q_generation_time_end - q_generation_time_start


        print(f"Finished creating evaluation data for chunk {doc+1}")
        print(f"Generation time for doc: {total_elapsed_time_generation}")
        print("---")
        
    generation_time_end = time.time()
    total_elapsed_time= generation_time_end - generation_time_start
    print(f"Generation time for all docs: {total_elapsed_time}")
        
    return dataset

In [ ]:
dataset_df = generate_dataset(docs_subset, llm, dataset)

num_questions_generated = dataset_df.shape[0]
print(f"Generated a total of {num_questions_generated} questions.")

In [ ]:
# display the first rows of the generated dataset
dataset_df.head()

## 8. Assessing the questions quality using Critique Agents
Critique agents are a technique used in natural language processing (NLP) to evaluate the quality and suitability of questions in a dataset for a particular task or application. In this case, the critique agents are employed to assess whether the questions in a dataset are valid for a Retrieval-Augmented Generation (RAG) system, which is a type of language model that combines information retrieval and generation capabilities.

The two main metrics evaluated by the critique agents are relevance and groundness.

Relevance

Relevance measures how useful and applicable a question is for a specific domain or context. In the context of financial and business analysis, the relevance prompt evaluates questions based on the following criteria:

- Is the question directly relevant to the work of financial and business analysts on Wall Street?
- Does the question address a practical problem or use case that analysts might encounter?
- Is the question clear and well-defined, avoiding ambiguity or vagueness?
- Does the question require a substantive answer that demonstrates understanding of financial topics?
- Would answering the question provide insights or knowledge that could be applied to real-world company evaluation tasks?

The relevance score ranges from 1 to 5, with a higher score indicating greater relevance and usefulness for financial and business analysts.

Groundness

Groundness measures how well a question can be answered based on the provided context or information. The groundness prompt evaluates questions based on the following criteria:

- Can the question be answered using only the information provided in the given context?
- Does the context provide little, some, substantial, or all the information needed to answer the question?

The groundness score also ranges from 1 to 5, with the following interpretations:

1. The question cannot be answered at all based on the given context.
2. The context provides very little relevant information to answer the question.
3. The context provides some relevant information to partially answer the question.
4. The context provides substantial information to answer most aspects of the question.
5. The context provides all the information needed to fully and unambiguously answer the question.

By evaluating both relevance and groundness, the critique agents can help identify questions in the dataset that are well-suited for the RAG system, as well as those that may need to be revised, removed, or supplemented with additional context or information.

In [ ]:
groundness_check_prompt_template = PromptTemplate(
    input_variables=["context","question"],
    template="""

    <Instructions Structure>
    1. Provide context
    2. Ask for evaluation/reasoning
    3. Ask for total rating score 
    </Instructions Structure>

    <Instructions>
    You will be given a context and a question related to that context.

    Your task is to provide an evaluation of how well the given question can be answered using only the information provided in the context. Rate this on a scale from 1 to 5, where:

    1 = The question cannot be answered at all based on the given context
    2 = The context provides very little relevant information to answer the question
    3 = The context provides some relevant information to partially answer the question 
    4 = The context provides substantial information to answer most aspects of the question
    5 = The context provides all the information needed to fully and unambiguously answer the question

    First, read through the provided context carefully:

    <context>
    {context}
    </context>

    Then read the question:

    <question>
    {question}
    </question>

    Evaluate how well you think the question can be answered using only the context information. Provide your reasoning first in an <evaluation> section, explaining what relevant or missing information from the context led you to your evaluation score in only one sentence.

    Provide your evaluation in the following format:

    <rating>(Your rating from 1 to 5)</rating>
    
    <evaluation>(Your evaluation and reasoning for the rating)</evaluation>


    </Instructions>
    
    """)

relevance_check_prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""
    <Instructions Structure>
    1. Explain the task to the AI assistant
    2. Provide guidelines for evaluating the usefulness of the question
    3. Ask the AI to provide its evaluation and rating, with the rating first followed by the justification in only one sentence
    </Instructions Structure>

    <Instructions>
    You will be given a question related to Amazon Shareholder letters. Your task is to evaluate how useful this question would be for a financial and business analyst working in wallstreat.

    To evaluate the usefulness of the question, consider the following criteria:

    1. Relevance: Is the question directly relevant to your work? Questions that are too broad or unrelated to this domain should receive a lower rating.

    2. Practicality: Does the question address a practical problem or use case that analysts might encounter? Theoretical or overly academic questions may be less useful.

    3. Clarity: Is the question clear and well-defined? Ambiguous or vague questions are less useful.

    4. Depth: Does the question require a substantive answer that demonstrates understanding of financila topics? Surface-level questions may be less useful.

    5. Applicability: Would answering this question provide insights or knowledge that could be applied to real-world company evaluation tasks? Questions with limited applicability should receive a lower rating.

    Provide your evaluation in the following format:

    <rating>(Your rating from 1 to 5)</rating>
    
    <evaluation>(Your evaluation and reasoning for the rating)</evaluation>

    Here is the question:

    {question}
    </Instructions>
    """)

In [ ]:
def generate_groundness_check(question, source_raw): 
    # Pass in values to the input variables
    groundness_prompt = groundness_check_prompt_template.format(question=question, context=source_raw)
    
    groundness_rating = llm.invoke(groundness_prompt)
        
    return groundness_rating

def generate_relevance_check(question): 
    # Pass in values to the input variables
    relevance_prompt = relevance_check_prompt_template.format(question=question)
    
    relevance_rating = llm.invoke(relevance_prompt)
        
    return relevance_rating

In [ ]:
# Evaluating one of the generated questions for groundness and relevance
groundness_rating = generate_groundness_check(dataset_df.question[0], dataset_df.source_raw[0])
relevance_rating = generate_relevance_check(dataset_df.question[0])

print("Groundness Score:")
print(groundness_rating.content)

print("---")

print("Relevance Score:")
print(relevance_rating.content)


In [ ]:
import re
# Helper functions to extract values from the string response by the LLM Critique Agents.
def extract_rating(text):
    pattern = r'<rating>(.*?)</rating>'
    match = re.search(pattern, text)
    if match:
        rating = match.group(1)
        return rating
    else:
        return None
    
def extract_reasoning(text):
    pattern = r'<evaluation>(.*?)</evaluation>'
    match = re.search(pattern, text)
    if match:
        rating = match.group(1)
        return rating
    else:
        return None

In [ ]:
def evaluate_dataset(dataset):
    for index, row in dataset.iterrows():

        question = row['question']
        source_raw = row['source_raw']

        # Generate groundness check
        groundness_check = generate_groundness_check(question, source_raw)
        groundness_score = extract_rating(groundness_check.content)
        groundness_score_reasoning = extract_reasoning(groundness_check.content)

        dataset.at[index, 'groundness_score'] = groundness_score
        dataset.at[index, 'groundness_score_reasoning'] = groundness_score_reasoning

        # Generate relevance check
        relevance_check = generate_relevance_check(question)
        relevancy_score = extract_rating(relevance_check.content)
        relevancy_score_reasoning = extract_reasoning(relevance_check.content)

        dataset.at[index, 'relevancy_score'] = relevancy_score
        dataset.at[index, 'relevancy_score_reasoning'] = relevancy_score_reasoning

    return dataset

Now that the concept of critique agents has been established including the prompt for groundness and relevance scores you iterate over the generated dataset dataset and assign each question a score. Depending on your needs you can eliminate questions with a score beneath a certain threshold from the dataset.

In [ ]:
dataset_evaluated = evaluate_dataset(dataset_df)

In [ ]:
dataset_evaluated.head()

### Conclusion

Generating synthetic datasets is a powerful technique for evaluating retrieval augmented generation (RAG) systems, particularly in the early stages of development when real-world data is scarce or difficult to obtain. By leveraging large language models and knowledge retrieval context, this approach enables the creation of diverse, realistic, and representative datasets that mimic real human interactions.

Throughout this notebook, you have explored the process of generating a synthetic dataset for a QA-RAG application using Anthropic's Claude via the Bedrock API, Python, and Langchain. You covered essential steps, including setting up the environment, loading and preparing context data, initial question generation, answer generation, extracting relevant context, evolving questions to fit end-user behavior, automated dataset generation, and assessing question quality.

While this approach offers numerous benefits, it is essential to acknowledge its limitations. First, the quality of the synthetic dataset heavily relies on the performance and capabilities of the underlying language model and knowledge retrieval system. Biases and limitations present in these components may be reflected in the generated dataset. Additionally, capturing the full complexity and nuances of real-world interactions can be challenging, as synthetic datasets may not account for all edge cases or unexpected scenarios.

Despite these limitations, generating synthetic datasets remains a valuable tool for accelerating the development and evaluation of RAG systems. By streamlining the evaluation process and enabling iterative development cycles, this approach can contribute to the creation of better-performing AI systems.

We encourage developers, researchers, and enthusiasts to explore the open-source tools like RAGAS mentioned in this notebook and experiment with generating synthetic datasets for their own RAG applications. Hands-on experience with this technique can provide valuable insights and contribute to the advancement of RAG systems in various domains.

Remember, synthetic dataset generation is not a silver bullet, but rather a powerful tool that should be used in conjunction with other evaluation techniques and real-world data when available. By embracing this approach and continuously improving upon it, you can accelerate the development of more robust and capable RAG systems, ultimately enhancing the user experience and unlocking new possibilities in natural language processing.